<a href="https://colab.research.google.com/github/bhuiyanmobasshir94/Cow-weight-and-Breed-Prediction/blob/main/notebooks/027_dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import sys
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import pathlib

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
images_dataset_url = "https://cv-datasets-2021.s3.amazonaws.com/images.tar.gz"
images_data_dir = tf.keras.utils.get_file(origin=images_dataset_url,
                                   fname='images',
                                   untar=True)
images_data_dir = pathlib.Path(images_data_dir)

833060864/833046463 [==============================] - 17s 0us/step


In [ ]:
yt_images_dataset_url = "https://cv-datasets-2021.s3.amazonaws.com/yt_images.tar.gz"
yt_images_data_dir = tf.keras.utils.get_file(origin=yt_images_dataset_url,
                                   fname='yt_images',
                                   untar=True)
yt_images_data_dir = pathlib.Path(yt_images_data_dir)

 292052992/4026992587 [=>............................] - ETA: 47s

In [ ]:
if sys.platform == 'darwin':
  os.system(f"dot_clean {images_data_dir}")
  os.system(f"dot_clean {yt_images_data_dir}")
elif sys.platform.startswith("lin"):
  os.system(f"cd {images_data_dir} && find . -type f -name '._*' -delete")
  os.system(f"cd {yt_images_data_dir} && find . -type f -name '._*' -delete")


In [ ]:
image_count = len(list(images_data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
yt_image_count = len(list(yt_images_data_dir.glob('*/*.jpg')))
print(yt_image_count)

In [ ]:
df = pd.read_csv("https://cv-datasets-2021.s3.amazonaws.com/dataset.csv")
df.shape

In [ ]:
df.head(2)

In [ ]:
images = list(images_data_dir.glob('*/*.jpg'))
yt_images = list(yt_images_data_dir.glob('*/*.jpg'))

In [ ]:
min_height = 0
max_height = 0
min_width = 0
max_width = 0
for i, image in enumerate(images):
  if i == 0:
    min_height = h
    max_height = h
    min_width = w
    max_width = w

  w, h = PIL.Image.open(str(image)).size
  if h <= min_height:
    min_height = h
  if h >= max_height:
    max_height = h

  if w <= min_width:
    min_width = w
  if w >= max_width:
    max_width = w

print(f"min_height: {min_height}")
print(f"min_width: {min_width}")
print(f"max_height: {max_height}")
print(f"max_width: {max_width}")

In [ ]:
min_height = 0
max_height = 0
min_width = 0
max_width = 0
for i, image in enumerate(yt_images):
  if i == 0:
    min_height = h
    max_height = h
    min_width = w
    max_width = w

  w, h = PIL.Image.open(str(image)).size
  if h <= min_height:
    min_height = h
  if h >= max_height:
    max_height = h

  if w <= min_width:
    min_width = w
  if w >= max_width:
    max_width = w

print(f"min_height: {min_height}")
print(f"min_width: {min_width}")
print(f"max_height: {max_height}")
print(f"max_width: {max_width}")

In [ ]:
f_df = pd.DataFrame(columns = ['file_path', 'teeth', 'age_in_year', 'breed', 'height_in_inch', 'weight_in_kg'])
for index, row in df.iterrows():
  images = list(images_data_dir.glob(f"{row['sku']}/*.jpg"))
  yt_images = list(yt_images_data_dir.glob(f"{row['sku']}/*.jpg"))

  for image in images:
    f_df = f_df.append({'file_path' : image, 'teeth' : row['teeth'], 'age_in_year' : row['age_in_year'], 'breed': row['breed'], 'height_in_inch': row['height_in_inch'], 'weight_in_kg': row['weight_in_kg']}, 
                ignore_index = True)
    
  for image in yt_images:
    f_df = f_df.append({'file_path' : image, 'teeth' : row['teeth'], 'age_in_year' : row['age_in_year'], 'breed': row['breed'], 'height_in_inch': row['height_in_inch'], 'weight_in_kg': row['weight_in_kg']}, 
                ignore_index = True)


In [ ]:
f_df.shape

In [ ]:
f_df.head(1)

In [ ]:
train_df, valid_test_df = train_test_split(f_df, test_size=0.3)
validation_df, test_df = train_test_split(valid_test_df, test_size=0.3)
print(f"train_df: {train_df.shape}")
print(f"validation_df: {validation_df.shape}")
print(f"test_df: {test_df.shape}")

In [ ]:
# min_height: 450
# min_width: 800

# input: [image, teeth]
# outpur: [age_in_year, breed, height_in_inch, weight_in_kg]

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(450, 800, 3), # (input_height, input_width, input_channel)
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        # self.n_teeth = df[X_col['teeth']].max()
        # self.n_breed = df[y_col['breed']].nunique()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, target_size):
        image = tf.keras.preprocessing.image.load_img(path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        # image_arr = image_arr[ymin:ymin+h, xmin:xmin+w]
        image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy()

        return image_arr/255.
    
    def __get_output(self, label, num_classes):
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['file_path']]       
        # teeth_batch = batches[self.X_col['teeth']]

        # breed_batch = batches[self.y_col['breed']]
        weight_in_kg_batch = batches[self.y_col['weight_in_kg']]
        height_in_inch_batch = batches[self.y_col['height_in_inch']]
        age_in_year_batch = batches[self.y_col['age_in_year']]

        X0 = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

        # y0_batch = np.asarray([self.__get_output(y, self.n_teeth) for y in teeth_batch])
        # y1_batch = np.asarray([self.__get_output(y, self.n_breed) for y in breed_batch])

        y0 = np.asarray([tf.cast(y, tf.float32) for y in weight_in_kg_batch])
        y1 = np.asarray([tf.cast(y, tf.float32) for y in height_in_inch_batch])
        y2 = np.asarray([tf.cast(y, tf.float32) for y in age_in_year_batch])

        return X0, tuple([y0, y1, y2])
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
traingen = CustomDataGen(train_df,
                         X_col={'file_path':'file_path', 'teeth': 'teeth'},
                         y_col={'breed': 'breed', 'weight_in_kg': 'weight_in_kg', 'height_in_inch': 'height_in_inch', 'age_in_year': 'age_in_year'},
                         batch_size=128, input_size=(450, 800, 3))

In [ ]:
testgen = CustomDataGen(test_df,
                         X_col={'file_path':'file_path', 'teeth': 'teeth'},
                         y_col={'breed': 'breed', 'weight_in_kg': 'weight_in_kg', 'height_in_inch': 'height_in_inch', 'age_in_year': 'age_in_year'},
                         batch_size=128, input_size=(450, 800, 3))

In [ ]:
validgen = CustomDataGen(validation_df,
                         X_col={'file_path':'file_path', 'teeth': 'teeth'},
                         y_col={'breed': 'breed', 'weight_in_kg': 'weight_in_kg', 'height_in_inch': 'height_in_inch', 'age_in_year': 'age_in_year'},
                         batch_size=128, input_size=(450, 800, 3))

In [ ]:
input = keras.Input(shape=(450, 800, 3), name="original_img")
x = layers.Conv2D(128, 3, activation="relu")(input)
x = layers.Conv2D(256, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(256, 3, activation="relu")(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.GlobalMaxPooling2D()(x)

out_a = keras.layers.Dense(1, activation='linear', name='wt_rg')(x)
out_b = keras.layers.Dense(1, activation='linear', name='ht_rg')(x)
out_c = keras.layers.Dense(1, activation='linear', name='ag_rg')(x)

encoder = keras.Model( inputs = input, outputs = [out_a, out_b, out_c], name="encoder")

In [ ]:
encoder.compile(
    loss = {
        "wt_rg": tf.keras.losses.MeanSquaredError(),
        "ht_rg": tf.keras.losses.MeanSquaredError(),
        "ag_rg": tf.keras.losses.MeanSquaredError()
    },

    metrics = {
        "wt_rg": 'mse',
        "ht_rg": 'mse',
        "ag_rg": 'mse'
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [ ]:
y0 = None
y1 = None
y2 = None
x = None
for X, y in traingen:
  y0, y1, y2 = y
  x = X
  break
encoder.fit(x, [y0, y1, y2], epochs=1, verbose=2, batch_size=10, validation_split=0.2)